In [3]:
import pandas as pd
import glob
import os

files = glob.glob("../data/*.csv")   # or stock_data/*.csv — use your folder name

combined = []

for file in files:
    df = pd.read_csv(file)

    # extract only the ticker (just the filename)
    ticker = os.path.basename(file).replace(".csv", "")
    df["ticker"] = ticker  # now it's AAPL, GOOG, etc.

    df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
    df = df.dropna(subset=["Date"])
    combined.append(df)

merged_df = pd.concat(combined, ignore_index=True)
merged_df = merged_df.sort_values(["Date", "ticker"])

merged_df.to_csv("cleaned_stocks.csv", index=False)

merged_df.head()

,Date,Close,High,Low,Open,Volume,ticker
0,2009-01-02,2.721686,2.730385,2.554037,2.575630,746015200,AAPL
3774,2009-01-02,2.718000,2.726500,2.553500,2.567500,145928000,AMZN
7548,2009-01-02,7.948608,7.960977,7.557264,7.633949,144961322,GOOG
14245,2009-01-02,14.872976,14.924187,14.170662,14.287715,50084000,MSFT
18019,2009-01-02,0.199652,0.201027,0.184294,0.184982,497124000,NVDA


## SMA (Simple Moving Average)

In [4]:
# Calculate 20-day and 50-day SMA
merged_df["SMA_20"] = merged_df.groupby("ticker")["Close"].transform(lambda x: x.rolling(window=20).mean())
merged_df["SMA_50"] = merged_df.groupby("ticker")["Close"].transform(lambda x: x.rolling(window=50).mean())

# Preview
merged_df[["Date","ticker","Close","SMA_20","SMA_50"]].head(10)


,Date,ticker,Close,SMA_20,SMA_50
0,2009-01-02,AAPL,2.721686,NaN,NaN
3774,2009-01-02,AMZN,2.718000,NaN,NaN
7548,2009-01-02,GOOG,7.948608,NaN,NaN
14245,2009-01-02,MSFT,14.872976,NaN,NaN
18019,2009-01-02,NVDA,0.199652,NaN,NaN
1,2009-01-05,AAPL,2.836553,NaN,NaN
3775,2009-01-05,AMZN,2.703000,NaN,NaN
7549,2009-01-05,GOOG,8.115089,NaN,NaN
14246,2009-01-05,MSFT,15.011971,NaN,NaN
18020,2009-01-05,NVDA,0.203319,NaN,NaN


## Relative Strength Index

In [5]:
def compute_rsi(series, period=14):
    delta = series.diff()

    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)

    avg_gain = gain.rolling(period).mean()
    avg_loss = loss.rolling(period).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

# Apply RSI per ticker
merged_df["RSI_14"] = merged_df.groupby("ticker")["Close"].transform(lambda x: compute_rsi(x))


In [6]:
merged_df[["Date","ticker","Close","RSI_14"]].head(20)

,Date,ticker,Close,RSI_14
0,2009-01-02,AAPL,2.721686,NaN
3774,2009-01-02,AMZN,2.718000,NaN
7548,2009-01-02,GOOG,7.948608,NaN
14245,2009-01-02,MSFT,14.872976,NaN
18019,2009-01-02,NVDA,0.199652,NaN
1,2009-01-05,AAPL,2.836553,NaN
3775,2009-01-05,AMZN,2.703000,NaN
7549,2009-01-05,GOOG,8.115089,NaN
14246,2009-01-05,MSFT,15.011971,NaN
18020,2009-01-05,NVDA,0.203319,NaN
